In [48]:
%matplotlib inline

from IPython import display
import pylab as pl
from matplotlib.patches import Ellipse
from sklearn import datasets
import numpy as np
import matplotlib.pyplot
import matplotlib.pyplot as plt
import time
from sklearn import datasets
import random
import math
import pandas as pd
import random
import matplotlib.pyplot
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc as gc

In [49]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

wine=load_wine()
df = pd.DataFrame(wine['data'],columns=wine['feature_names'])
label=wine["target"]

In [50]:
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn import metrics

X_train,X_test,y_train,y_test=train_test_split(df,label,test_size=0.3,random_state=50)
model=LogisticRegression()
model.fit(X_train,y_train)
predicted=model.predict(X_test)
score=metrics.accuracy_score(y_test,predicted)
print(score)

0.9629629629629629


In [51]:
def initialize(n_feat,sz):
    population=[]
    n_feat=10
    sz=100
    for i in range(sz):
        chromosome=np.ones(n_feat,dtype=int)
        chromosome[:int(0.3*n_feat)]=0
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population

In [52]:
def rastrigin_fitness_fun(population,n_feat):
    A=10
    scores=[]
    i=0
    for chromosome in range(len(population)):
        score=np.sum(population[chromosome]**2-A*np.cos(2*np.pi*population[chromosome]))
        score+=A*n_feat
        scores.append(score)
    return scores,population  

In [53]:
def select(scores,best_fit):
    selected_indiviuals_set=set()
    total_fitness=sum(scores)
    selection_prob=[fitness/total_fitness for fitness in scores]
    r=random.uniform(0,1/len(best_fit))
    prob_sum=0.0
    for i,x in enumerate(best_fit):
        prob_sum+=selection_prob[i]
        while r<prob_sum:
            r+=1/len(best_fit)
            selected_indiviuals_set.add(tuple(best_fit[i]))
    selected_indiviuals_list=[np.array(selected_inividual) for selected_inividual in selected_indiviuals_set]
    return selected_indiviuals_list
    
    

In [54]:
def getMask(n_feat):
    mask=[0]*n_feat
    for k in range(n_feat):
        if(random.uniform(0,1)<=0.5):
            mask[k]=1
    return mask
    

In [55]:
def crossover(pop_selc,n_feat):
    mask=getMask(n_feat)
    pop_next_gen=[]
    sz=len(pop_selc)
    for i in range(sz):
        j=random.randint(0,sz-1)
        check_j=i+1
        if j==i and  check_j<sz:
            j=check_j
        else:
            j=i-1
        p1=pop_selc[i].copy()
        p2=pop_selc[j].copy()
        c1,c2=p1.copy(),p2.copy()
        for k in range(len(mask)):
            if mask[k] == 1:
                temp=c1[k]
                c1[k]=c2[k]
                c2[k]=temp
                pop_next_gen.append(c1)
                pop_next_gen.append(c2)
    return pop_next_gen

In [56]:
def mutation(pop_next_gen,n_feat,mutation_rate,std_dev):
    mutated_pop=[]
    for i in range(len(pop_next_gen)):
        chromosome=pop_next_gen[i].copy()
        for j in range(n_feat):
            if(random.random()<mutation_rate):
                mutate=np.random.normal(loc=0,scale=std_dev)
                chromosome[j]+=mutate
        mutated_pop.append(chromosome)
    return mutated_pop
    

In [67]:
def generations(sz,n_feat,mutation_rate,n_gens):
    chromosomes_pop=initialize(n_feat,sz)
    stats_min = np.zeros(n_gens)
    stats_avg = np.zeros(n_gens)
    stats_max = np.zeros(n_gens)
    for i in range(n_gens):
        scores,best_fit=rastrigin_fitness_fun(chromosomes_pop,n_feat)
        best_chromo=scores[0]
        #print("!===========!",best_chromo)
        best_score=best_fit[0]
        #print("!===========!",best_score)
        stats_min[i]=np.min(scores)
        #print("!===========!",stats_min)
        stats_max[i]=np.amax(scores)
        #print("!===========!",stats_max)
        stats_avg[i]=np.mean(scores)
        #print("!===========!",stats_avg)
        pop_selc=select(scores,best_fit)
        pop_cross_res=crossover(pop_selc,n_feat)
        chromosomes_pop=mutation(pop_cross_res,n_feat,mutation_rate,0.3)
    return best_chromo,best_score,stats_min,stats_max,stats_avg

In [106]:
if __name__ == "__main__":
    best_chromo,best_score,stats_min,stats_max,stats_avg=generations(30,10,0.01,58)
    # plot the statistics
    plt.plot(stats_min,'r')
    plt.plot(stats_avg,'b')
    plt.plot(stats_max,'g')
    plt.ylabel('accuracy')
    plt.xlabel('generations')
    plt.show()
    gc.collect()

KeyboardInterrupt: 